<a href="https://colab.research.google.com/github/diptoromeo/Research-paper-classification-and-recommendation-system-with-BERT-and-CNN-BiLSTM-/blob/main/ELMo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import random

import keras.models
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import os
import pandas as pd
import regex
import numpy as np
from sklearn.model_selection import train_test_split
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from string import punctuation
import tensorflow as tf
from keras.models import Model
from keras.layers import Dense, Lambda, Input
import tensorflow_hub as hub
import tensorflow.compat.v1 as tf
tf.disable_eager_execution()
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

from tensorflow.python.client import device_lib

import timeit

print(tf.__version__)



print(device_lib.list_local_devices())


2.14.0
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 12366125872142117970
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 14357954560
locality {
  bus_id: 1
  links {
  }
}
incarnation: 12869870668983119989
physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
xla_global_id: 416903419
]


In [10]:
##===============================Nltk abstract_words tokenize======================================
with open('/content/FGFSJournal.txt', 'rt', encoding='UTF8') as file:
    FGFS_abstract = []
    for line in file:
        if '<abstract>' in line:
            abstract = line.split('</abstract>')[0].split('<abstract>')[-1]
            abstract = ''.join(i for i in abstract if not i.isdigit()).lower()
            FGFS_abstract.append(abstract)

print(len(FGFS_abstract))

5659


In [13]:
random.shuffle(FGFS_abstract)
train_data = FGFS_abstract[:4527]
test_data = FGFS_abstract[4527:]
print("train data:", len(train_data))
print("test data:", len(test_data))

train, valid = train_test_split(train_data, test_size=0.25, random_state=42)
print("train:", len(train))
print("valid:", len(valid))

train data: 4527
test data: 1132
train: 3395
valid: 1132


In [21]:
#======================================================train_labels==========================================================================
ten_words = ['paper', 'system', 'performance', 'network', 'model', 'service', 'time', 'information', 'approach', 'cloud']
twenty_words = ['paper', 'system', 'performance', 'network', 'model', 'service', 'time', 'information', 'approach', 'cloud',
                'problem', 'process', 'security', 'analysis', 'application', 'method', 'research', 'framework', 'number', 'resource']
thirty_words = ['paper', 'system', 'performance', 'network', 'model', 'service', 'time', 'information', 'approach', 'cloud',
                'problem', 'process', 'security', 'analysis', 'application', 'method', 'research', 'framework', 'number', 'resource',
               'environment', 'algorithm', 'energy', 'management', 'architecture', 'access', 'scheme', 'communication', 'execution', 'order']

train_count = 2
valid_count = 3
test_count = 3

train10_labels = []
for i in range(0, 3395):
    count = 0
    for j in range(0, len(ten_words)):
      if ten_words[j] in train[i]:
            count += 1
    if count >=train_count:
        train10_labels.append(1)
    else:
        train10_labels.append(0)


##====valid labels==================
valid10_labels = []
for i in range(0, 1132): #1132

    count = 0
    for j in range(0, len(ten_words)):
        if ten_words[j] in valid[i]:
            count += 1
    if count >=valid_count:
        valid10_labels.append(1)
    else:
        valid10_labels.append(0)


##====test labels====================
test10_labels = []
for i in range(0, 1132):

    count = 0
    for j in range(0, len(ten_words)):
        if ten_words[j] in test_data[i]:
            count += 1
    if count >=test_count:
        test10_labels.append(1)
    else:
        test10_labels.append(0)


#====================================20wordstrain_labels====================================
train_count = 3
valid_count = 4
test_count = 4

train20_labels = []
for i in range(0, 3395):
    count = 0
    for j in range(0, len(twenty_words)):
        if twenty_words[j] in train[i]:
            count += 1
    if count >=train_count:
        train20_labels.append(1)
    else:
        train20_labels.append(0)


##====valid labels============================
valid20_labels = []
for i in range(0, 1132): #1132

    count = 0
    for j in range(0, len(twenty_words)):
        if twenty_words[j] in valid[i]:
            count += 1
    if count >=valid_count:
        valid20_labels.append(1)
    else:
        valid20_labels.append(0)


##===test labels==============================
test20_labels = []
for i in range(0, 1132):

    count = 0
    for j in range(0, len(twenty_words)):
        if twenty_words[j] in test_data[i]:
            count += 1
    if count >=test_count:
        test20_labels.append(1)
    else:
        test20_labels.append(0)


#====================================30Words_train_labels====================================
train_count = 3
valid_count = 4
test_count = 4

train30_labels = []
for i in range(0, 3395):
    count = 0
    for j in range(0, len(thirty_words)):
        if thirty_words[j] in train[i]:
            count += 1
    if count >=train_count:
        train30_labels.append(1)
    else:
        train30_labels.append(0)


##====valid labels==========================
valid30_labels = []
for i in range(0, 1132): #1132

    count = 0
    for j in range(0, len(thirty_words)):
        if thirty_words[j] in valid[i]:
            count += 1
    if count >=valid_count:
        valid30_labels.append(1)
    else:
        valid30_labels.append(0)


##====test labels==========================
test30_labels = []
for i in range(0, 1132):

    count = 0
    for j in range(0, len(thirty_words)):
        if thirty_words[j] in test_data[i]:
            count += 1
    if count >=test_count:
        test30_labels.append(1)
    else:
        test30_labels.append(0)

In [15]:
## =============== get elmo from tensorflow hub==============================================
## Load pre trained ELMo model
elmo = hub.Module("https://tfhub.dev/google/elmo/3", trainable=True)

## create an instance of ELMo
## 데이터의 이동이 케라스 → 텐서플로우 → 케라스가 되도록 하는 함수
def ELMoEmbedding(x):
    return elmo(tf.squeeze # 차원 중 사이즈 1인 것을 찾아서 재거한다.
                (tf.cast # tf.cast()는 배열의 dtype을 변경해 준다.
                (x, tf.string)),
                as_dict=True, # 여러 입력이 있는 경우 as_dict를 통해 전달해야 한다.
                signature="default")["default"] # signature 선택 사항에 이름을 전달하여 선택할 수 있습니다.

In [22]:
X_train, x_valid = train_test_split(train_data, test_size=0.25, random_state=42)

X_train = np.asarray(X_train)
y10_train = np.asarray(train10_labels)
y20_train = np.asarray(train20_labels)
y30_train = np.asarray(train30_labels)
x_valid = np.asarray(x_valid)
y10_valid = np.asarray(valid10_labels)
y20_valid = np.asarray(valid20_labels)
y30_valid = np.asarray(valid30_labels)
X_test = np.asarray(test_data)
y10_test = np.asarray(test10_labels)
y20_test = np.asarray(test20_labels)
y30_test = np.asarray(test30_labels)

In [17]:
###===============================model build=============================
epochs = 10
batch_sizes = 16
validation_splits = 0.33


# Build Model
input_text = Input(shape=(1,), dtype=tf.string)
embedding = Lambda(ELMoEmbedding, output_shape=(1024, ))(input_text)
dense = Dense(128, activation='relu')(embedding)
pred = Dense(1, activation='softmax')(dense)
model = Model(inputs=[input_text], outputs=pred)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

##Calling `save('my_model')` creates a SavedModel folder `my_model`.
model.save("ElMo_model.h5")

# It can be used to reconstruct the model identically.
ElMo_model = keras.models.load_model("ElMo_model.h5")

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    history10 = model.fit(X_train, y10_train, validation_data=(x_valid, y10_valid), epochs=epochs, batch_size=batch_sizes)

    ELMo10_Accuracy = model.evaluate(X_train, y10_train, verbose=1)
    print(('ELMo_Accuracy: %f' % (ELMo10_Accuracy[1] * 100)))


hist_df = pd.DataFrame(history10.history)
hist_df.to_excel("ELMo_train_test_acc.xlsx", index=False)



Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 lambda (Lambda)             (None, 1024)              0         
                                                                 
 dense (Dense)               (None, 128)               131200    
                                                                 
 dense_1 (Dense)             (None, 1)                 129       
                                                                 
Total params: 131329 (513.00 KB)
Trainable params: 131329 (513.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Train on 3395 samples, validate on 1132 samples
Epoch 1/10
3395/3395 [==============================] - ETA: 0s - loss: 0.2767 - accuracy: 0.9108

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training_v1.py:2335: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


3395/3395 [==============================] - 431s 127ms/sample - loss: 0.2767 - accuracy: 0.9108 - val_loss: 0.5572 - val_accuracy: 0.7553
Epoch 2/10
3395/3395 [==============================] - 294s 87ms/sample - loss: 0.2481 - accuracy: 0.9108 - val_loss: 0.6265 - val_accuracy: 0.7553
Epoch 3/10
3395/3395 [==============================] - 285s 84ms/sample - loss: 0.2359 - accuracy: 0.9108 - val_loss: 0.6097 - val_accuracy: 0.7553
Epoch 4/10
3395/3395 [==============================] - 283s 83ms/sample - loss: 0.2283 - accuracy: 0.9108 - val_loss: 0.7134 - val_accuracy: 0.7553
Epoch 5/10
3395/3395 [==============================] - 282s 83ms/sample - loss: 0.2214 - accuracy: 0.9108 - val_loss: 0.7160 - val_accuracy: 0.7553
Epoch 6/10
3395/3395 [==============================] - 283s 83ms/sample - loss: 0.2126 - accuracy: 0.9108 - val_loss: 0.6297 - val_accuracy: 0.7553
Epoch 7/10
3395/3395 [==============================] - 281s 83ms/sample - loss: 0.2049 - accuracy: 0.9108 - val_los

In [18]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    history20 = model.fit(X_train, y20_train, validation_data=(x_valid, y20_valid), epochs=epochs, batch_size=batch_sizes)

    ELMo20_Accuracy = model.evaluate(X_train, y20_train, verbose=1)
    print(('ELMo_Accuracy: %f' % (ELMo20_Accuracy[1] * 100)))

hist_df = pd.DataFrame(history20.history)
hist_df.to_excel("ELMo20_train_test_acc.xlsx", index=False)


Train on 3395 samples, validate on 1132 samples
Epoch 1/10
3395/3395 [==============================] - 300s 88ms/sample - loss: 0.1961 - accuracy: 0.9432 - val_loss: 0.4516 - val_accuracy: 0.8763
Epoch 2/10
3395/3395 [==============================] - 288s 85ms/sample - loss: 0.1738 - accuracy: 0.9432 - val_loss: 0.4360 - val_accuracy: 0.8763
Epoch 3/10
3395/3395 [==============================] - 288s 85ms/sample - loss: 0.1610 - accuracy: 0.9432 - val_loss: 0.4783 - val_accuracy: 0.8763
Epoch 4/10
3395/3395 [==============================] - 288s 85ms/sample - loss: 0.1583 - accuracy: 0.9432 - val_loss: 0.3387 - val_accuracy: 0.8763
Epoch 5/10
3395/3395 [==============================] - 287s 85ms/sample - loss: 0.1466 - accuracy: 0.9432 - val_loss: 0.3648 - val_accuracy: 0.8763
Epoch 6/10
3395/3395 [==============================] - 283s 83ms/sample - loss: 0.1420 - accuracy: 0.9432 - val_loss: 0.3592 - val_accuracy: 0.8763
Epoch 7/10
3395/3395 [==============================] - 28

In [23]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    history30 = model.fit(X_train, y30_train, validation_data=(x_valid, y30_valid), epochs=epochs, batch_size=batch_sizes)

    ELMo30_Accuracy = model.evaluate(X_train, y30_train, verbose=1)
    print(('ELMo_Accuracy: %f' % (ELMo30_Accuracy[1] * 100)))


hist_df = pd.DataFrame(history30.history)
hist_df.to_excel("ELMo30_train_test_acc.xlsx", index=False)

Train on 3395 samples, validate on 1132 samples
Epoch 1/10
3395/3395 [==============================] - 295s 87ms/sample - loss: 0.1090 - accuracy: 0.9723 - val_loss: 0.2389 - val_accuracy: 0.9390
Epoch 2/10
3395/3395 [==============================] - 288s 85ms/sample - loss: 0.0914 - accuracy: 0.9723 - val_loss: 0.2377 - val_accuracy: 0.9390
Epoch 3/10
3395/3395 [==============================] - 285s 84ms/sample - loss: 0.0876 - accuracy: 0.9723 - val_loss: 0.1829 - val_accuracy: 0.9390
Epoch 4/10
3395/3395 [==============================] - 280s 83ms/sample - loss: 0.0751 - accuracy: 0.9723 - val_loss: 0.2354 - val_accuracy: 0.9390
Epoch 5/10
3395/3395 [==============================] - 282s 83ms/sample - loss: 0.0760 - accuracy: 0.9723 - val_loss: 0.2009 - val_accuracy: 0.9390
Epoch 6/10
3395/3395 [==============================] - 282s 83ms/sample - loss: 0.0751 - accuracy: 0.9723 - val_loss: 0.2417 - val_accuracy: 0.9390
Epoch 7/10
3395/3395 [==============================] - 28

In [24]:
##===============================ELMoModel Evaluation========================================
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    # Here X_test, y_test are the test data points
    predictions = ElMo_model.predict(X_test)
    predictions = predictions[:, 0]

    # CBOW Tst Accuracy
    ELMO_Test_Accuracy = accuracy_score(y10_test, np.round(abs(predictions)))
    print('ELMO_Test_Accuracy: %f' % ELMO_Test_Accuracy)

    # Calculating the precision score of classifier
    ELMO_Precision = precision_score(y10_test, np.round(abs(predictions)))
    print('ELMO_Precision: %f' % ELMO_Precision)

    # Calculating the recall score of classifier
    ELMO_Recall = recall_score(y10_test, np.round(abs(predictions)))
    print('ELMO_Recall: %f' % ELMO_Recall)

    # Calculating the F1 score of classifier
    ELMO_F1_Score = f1_score(y10_test, np.round(abs(predictions)))
    print('ELMO_F1_Score: %f' % ELMO_F1_Score)




data = {
    'Name': ["ELMo_Accuracy", "ELMO_Test_Accuracy", "ELMO_Precision", "ELMO_Recall", "ELMO_F1_Score"],
    'Scores': [ELMo10_Accuracy, ELMO_Test_Accuracy, ELMO_Precision, ELMO_Recall, ELMO_F1_Score] #
}
df = pd.DataFrame(data)
df.to_excel('ELMo_Scores_file.xlsx', index=False)




/usr/local/lib/python3.10/dist-packages/keras/src/engine/training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


ELMO_Test_Accuracy: 0.757951
ELMO_Precision: 0.757951
ELMO_Recall: 1.000000
ELMO_F1_Score: 0.862312


In [25]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    # Here X_test, y_test are the test data points
    predictions = ElMo_model.predict(X_test)
    predictions = predictions[:, 0]

    # CBOW Tst Accuracy
    ELMO20_Test_Accuracy = accuracy_score(y20_test, np.round(abs(predictions)))
    print('ELMO20_Test_Accuracy: %f' % ELMO20_Test_Accuracy)

    # Calculating the precision score of classifier
    ELMO20_Precision = precision_score(y20_test, np.round(abs(predictions)))
    print('ELMO20_Precision: %f' % ELMO20_Precision)

    # Calculating the recall score of classifier
    ELMO20_Recall = recall_score(y20_test, np.round(abs(predictions)))
    print('ELMO20_Recall: %f' % ELMO20_Recall)

    # Calculating the F1 score of classifier
    ELMO20_F1_Score = f1_score(y20_test, np.round(abs(predictions)))
    print('ELMO20_F1_Score: %f' % ELMO20_F1_Score)

data20 = {
    'Name': ["ELMo20_Accuracy", "ELMO20_Test_Accuracy", "ELMO20_Precision", "ELMO20_Recall", "ELMO20_F1_Score"],
    'Scores': [ELMo20_Accuracy, ELMO20_Test_Accuracy, ELMO20_Precision, ELMO20_Recall, ELMO20_F1_Score] #
}
df = pd.DataFrame(data20)
df.to_excel('ELMo20_Scores_file.xlsx', index=False)


ELMO20_Test_Accuracy: 0.854240
ELMO20_Precision: 0.854240
ELMO20_Recall: 1.000000
ELMO20_F1_Score: 0.921391


In [26]:

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    # Here X_test, y_test are the test data points
    predictions = ElMo_model.predict(X_test)
    predictions = predictions[:, 0]

    # CBOW Tst Accuracy
    ELMO30_Test_Accuracy = accuracy_score(y30_test, np.round(abs(predictions)))
    print('ELMO30_Test_Accuracy: %f' % ELMO30_Test_Accuracy)

    # Calculating the precision score of classifier
    ELMO30_Precision = precision_score(y30_test, np.round(abs(predictions)))
    print('ELMO30_Precision: %f' % ELMO30_Precision)

    # Calculating the recall score of classifier
    ELMO30_Recall = recall_score(y30_test, np.round(abs(predictions)))
    print('ELMO30_Recall: %f' % ELMO30_Recall)

    # Calculating the F1 score of classifier
    ELMO30_F1_Score = f1_score(y30_test, np.round(abs(predictions)))
    print('ELMO30_F1_Score: %f' % ELMO30_F1_Score)

data30 = {
    'Name': ["ELMo30_Accuracy", "ELMO30_Test_Accuracy", "ELMO30_Precision", "ELMO30_Recall", "ELMO30_F1_Score"],
    'Scores': [ELMo30_Accuracy, ELMO30_Test_Accuracy, ELMO30_Precision, ELMO30_Recall, ELMO30_F1_Score] #
}
df = pd.DataFrame(data30)
df.to_excel('ELMo30_Scores_file.xlsx', index=False)

ELMO30_Test_Accuracy: 0.930212
ELMO30_Precision: 0.930212
ELMO30_Recall: 1.000000
ELMO30_F1_Score: 0.963844
